In [117]:
# Convert shapefile coordinates to lat / long coordinates and
# then to GeoJSON object.
#
# https://leafletjs.com/examples/geojson/
#
# get lat/lng coordinates from shapefile polygon
# https://gis.stackexchange.com/questions/218774/how-to-extract-long-and-lat-from-geometry-column-in-geopandas?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
# https://automating-gis-processes.github.io/2016/Lesson3-point-in-polygon.html

Brevard County's property appraiser site and downloaded a zip file that contained the shapesfiles for the cities within Brevard.  The file name is Cities.zip

https://www.bcpao.us/PublicData.aspx

Save the file and extract it's contents.  There are 4 files but the one I am interested in is cities.shp

In [118]:
import geopandas as gpd
import shapely
import json

In [119]:
file = '/home/curtis/mypy/data/brevard/cities.shp'

In [120]:
cities = gpd.read_file(file)
cities['CITY_NAME']

0              Cocoa Beach
1                Rockledge
2          Satellite Beach
3     Indian Harbour Beach
4              Indialantic
5          Melbourne Beach
6                  Malabar
7              Palm Shores
8        Melbourne Village
9                    Cocoa
10          Grant-Valkaria
11          Cape Canaveral
12               Melbourne
13              Titusville
14               Rockledge
15               Rockledge
16          West Melbourne
17             Palm Shores
18               Melbourne
19               Melbourne
20          West Melbourne
21          West Melbourne
22                Palm Bay
23          West Melbourne
24                Palm Bay
Name: CITY_NAME, dtype: object

In [121]:
wm = cities[ cities['CITY_NAME'] == 'Indialantic']
wm_lat_lng = wm.to_crs(epsg=4326)

In [122]:
geojson_file = 'indialantic.js'

In [123]:
poly_lines = []
multi_lines = []
mpoly = []
for index, row in wm_lat_lng.iterrows():
    poly = []
    if isinstance(row.geometry, shapely.geometry.polygon.Polygon):
        for pt in list(row['geometry'].exterior.coords):
            point = []
            # for Leaflet & geoJSON use (lat, lng)
            point.append(pt[1]) # long coordinate
            point.append(pt[0]) # lat
            poly.append(point)
        poly_lines.append(poly)
    elif isinstance(row.geometry, shapely.geometry.MultiPolygon):
        for mp in row.geometry:
            mpoly = []
            for p in list(mp.exterior.coords):
                mp = []
                mp.append(p[1])
                mp.append(p[0])
                mpoly.append(mp)
            poly_lines.append(mpoly)

In [124]:
with open(geojson_file, 'w') as f:
    num_poly_lines = len(poly_lines)
    row_poly_lines = 0
    f.write('[\n')
    for pl in poly_lines:
        row_poly_lines = row_poly_lines + 1
        num_rows = len(pl)
        row_count = 0
        f.write('\t[\n')
        f.write('\t\t[\n')
        for pt in pl:
            row_count = row_count + 1
            if row_count != num_rows:
                f.write('\t\t\t[{},{}],\n'.format(pt[0],pt[1]) )
            else:
                f.write('\t\t\t[{},{}]\n'.format(pt[0],pt[1]) )  
        f.write('\t\t]\n')
        if row_poly_lines != num_poly_lines:
            f.write('\t],\n')
        else:
            f.write('\t]\n')
    f.write(']\n')  